# Datathon Decision

Pt:
Notebook usada para desenvolvimento do sistema de recomendação de vagas com base em currículos proposto na último Tech Challenge do curso de pós-graduação da FIAP em Análise de Dados.

En:
Notebook used for the development of a job possition recomendation system based on CV proposed in the last Tech Challenge of the FIAP postgraduate course in Data Analytics.

## Importanto libs (Importing Libs)

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

## Step 1: Carregando os dados (Loading the Data)

In [ ]:
# Loading JSON files
# Carregando arquivos JSON

# The JSON files were taken from Decision DB, but first they anonymized the data.
# Os arquivos JSON foram pegos da base de dados da Decision, porém antes eles deixaram os dados anônimos.

with open(r'applicants.json', encoding='utf-8') as f:
    applicants = json.load(f)

with open(r'vagas.json', encoding='utf-8') as f:
    jobs = json.load(f)

with open("prospects.json", encoding="utf-8") as f:
    prospects = json.load(f)


📄 Detalhes dos Dados | Data Details

- Os dados estão no formato **JSON**.  
  _The data is in **JSON** format._
  
- Todas as informações sensíveis (de clientes, candidatos e analistas) foram **anonimizadas**, utilizando nomes, números de telefone e e-mails aleatórios.  
  _All sensitive information (related to clients, candidates, and analysts) has been **anonymized** using random names, phone numbers, and email addresses._

---

📁 Sobre os Arquivos | About the Files

`vagas.json`
- Chaveado pelo **código da vaga**.  
  _Keyed by the **job opening code**._
  
- Contém informações da vaga no **ATS** (sistema de rastreamento de candidatos), divididas em:
  - Informações básicas | _Basic information_
  - Perfil da vaga | _Job profile_
  - Benefícios | _Benefits_

**Campos importantes | Key fields:**
- Indicação se é vaga **relacionada a SAP**  
  _Indication of whether the job is **SAP-related**_
- **Cliente solicitante** | _Requesting client_
- **Nível profissional** e **nível de idiomas** exigidos  
  _Required **professional level** and **language proficiency**_
- **Atividades principais** e **competências técnicas**  
  _Key responsibilities and required technical skills_

---

`prospects.json`
- Também chaveado pelo **código da vaga**.  
  _Also keyed by the **job code**._

- Contém todas as **prospecções** da vaga.  
  _Contains all **prospecting entries** related to the job._

**Cada entrada inclui | Each entry includes:**
- Código da prospecção | _Prospect code_
- Nome | _Name_
- Comentário | _Comments_
- Situação do candidato | _Candidate’s status_

---

`applicants.json`
- Chaveado pelo **código do candidato**.  
  _Keyed by the **candidate ID**._

- Contém dados:
  - Básicos | _Basic details_
  - Pessoais | _Personal data_
  - Profissionais | _Professional background_
  - Formação | _Educational background_
  - Currículo completo | _Full résumé_

**Campos importantes | Key fields:**
- **Nível acadêmico**, **inglês** e **espanhol**  
  _Academic level, English and Spanish proficiency_
- **Conhecimentos técnicos** | _Technical skills_
- **Área de atuação** | _Area of expertise_
- **CV completo** | _Full résumé_

---

🧪 Exemplo de Utilização | Example Use Case

A vaga **`10976`** (chave no `vagas.json`) possui **25 prospecções** (chave `10976` no `prospects.json`), entre as quais o candidato **“Sr. Thales Freitas”** (chave `41496` no `applicants.json`) foi **contratado**.  
_Job opening **`10976`** (key in `vagas.json`) has **25 prospects** (key `10976` in `prospects.json`), among which candidate **“Sr. Thales Freitas”** (key `41496` in `applicants.json`) was **hired**._


## Step 2: Extraindo os textos como habilidades e descrições (Extracting Text like Skills & Descriptions)

In [ ]:

# From applicants
# Dos aplicantes
def extract_applicant_skills(applicant):
    skills = applicant["informacoes_profissionais"].get("conhecimentos_tecnicos", "")
    cv = applicant.get("cv_pt", "")
    return skills + " " + cv.lower()  # merge and normalize

# From jobs
# Das vagas
def extract_job_requirements(job):
    skills = job["perfil_vaga"].get("competencia_tecnicas_e_comportamentais", "")
    activities = job["perfil_vaga"].get("principais_atividades", "")
    return skills.lower() + " " + activities.lower()


##  Step 3: Criando um dataset de similaridades (Creating Matching Dataset)

In [ ]:
# Criando função que removerá stop words e limpara o texto
# Creating function that will remove stop words and clean the text
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('portuguese'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text, language='portuguese')
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)

In [ ]:
# Getting the ids
# Obtendo os ids
applicant_ids = list(applicants.keys())
job_ids = list(jobs.keys())

In [ ]:
# Extracting and preprocessing texts
# Extraindo e limpando o texto
applicant_texts = [
    preprocess(extract_applicant_skills(applicants[aid]))
    for aid in applicant_ids
]

job_texts = [
    preprocess(extract_job_requirements(jobs[jid]))
    for jid in job_ids
]

In [ ]:
# Initializing a TF-IDF vectorizer, which is converting text into numerical form

# Inicializando um vetor TF-IDF, que está convertendo texto em uma forma numérica

# Creating a tool that is converting text into numbers based on word importance

# Criando uma ferramenta que está convertendo texto em números com base na importância das palavras
vectorizer = TfidfVectorizer()  

# Fitting the vectorizer on the job descriptions and transforming them into numerical vectors

# Ajustando o vetor TF-IDF nas descrições de vagas e transformando-as em vetores numéricos
job_vecs = vectorizer.fit_transform(job_texts)  
# The vectorizer is learning from the job descriptions, finding important words and how often they are appearing
# Then, it is converting these descriptions into numerical vectors that a machine learning model can understand

# O vetor está aprendendo com as descrições de vagas, encontrando palavras importantes e com que frequência elas estão aparecendo
# Em seguida, está convertendo essas descrições em vetores numéricos que um modelo de aprendizado de máquina pode entender

# Transforming the applicants' texts into the same numerical format using the same vectorizer

# Transformando os textos dos candidatos para o mesmo formato numérico usando o mesmo vetor TF-IDF
applicant_vecs = vectorizer.transform(applicant_texts) 

# Now, we are using the same learned vectorizer to convert the applicants' texts into vectors in the same way
# This is allowing us to compare the job descriptions and the applicant texts, since both are now in the same numerical form

# Agora, estamos usando o mesmo vetor aprendido para converter os textos dos candidatos em vetores da mesma maneira
# Isso está nos permitindo comparar as descrições de vagas e os textos dos candidatos, já que ambos estão no mesmo formato numérico

In [ ]:
# Calculating the cosine similarity between the applicant vectors and the job vectors
# A função cosine_similarity está calculando a similaridade entre os vetores dos candidatos e os vetores das vagas
similarity_matrix = cosine_similarity(applicant_vecs, job_vecs)

# O resultado é uma matriz que mostra o quão semelhantes são os textos dos candidatos em relação às vagas
# The result is a similarity matrix where each element represents how similar a specific applicant’s text is to a specific job description. A higher value means a higher similarity.

In [ ]:
# This was used to save the output from similarity matrix and reuse so we don't spending time running everytime that we need
# Isso foi usado para salvar o resultado da matriz de similaridade e reutilizá-la, para que não gastemos tempo rodando o cálculo toda vez que precisarmos
np.save('similarity_matrix', similarity_matrix)

## Step 4: Recomendando principais vagas para aplicantes (Recommending Top Jobs for an Applicant)

In [ ]:
# This function shows job recommendations for a specific applicant based on the similarity between their profile and the jobs
# Esta função mostra a recomendação de vagas para um candidato específico baseado na similaridade entre seus perfis e as vagas
def show_recommendations_for_applicant(applicant_index, top_n=5):
    # Getting the applicant's ID using the index in the list of applicant IDs
    # Obtendo o ID do aplinte utilizando o index
    applicant_id = applicant_ids[applicant_index]

    # Getting the applicant's information from the applicants dictionary
    # Obetendo as informações do candidato do arquivo JSON
    applicant = applicants.get(applicant_id, {})  

    # Extracting important details from the applicant's profile
    # Obtendo ingormações detalahas do perfil do aplicante
    name = applicant.get("infos_basicas", {}).get("nome", "N/A")
    area = applicant.get("informacoes_profissionais", {}).get("area_atuacao", "N/A")
    skills = applicant.get("informacoes_profissionais", {}).get("conhecimentos_tecnicos", "N/A")
    academic = applicant.get("formacao_e_idiomas", {}).get("nivel_academico", "N/A")
    english = applicant.get("formacao_e_idiomas", {}).get("nivel_ingles", "N/A")
    spanish = applicant.get("formacao_e_idiomas", {}).get("nivel_espanhol", "N/A")
    cv_excerpt = applicant.get("cv_pt", "").strip().replace("\n", " ")[:300] + "..."

    # Displaying the applicant's basic information
    # Mostrando as informações do aplicante
    print(f"\n=== 🧑 Applicant: {name} (ID: {applicant_id}) ===")
    print(f"Área de Atuação: {area}")
    print(f"Conhecimentos Técnicos: {skills}")
    print(f"Formação: {academic} | Inglês: {english} | Espanhol: {spanish}")
    print(f"📄 CV (resumo): {cv_excerpt}\n")
    print('\n--------------------------------------------------------------------\n')

    # Getting the similarity scores between this applicant and all the jobs
    # Obtendo o valor de similaridade entre os aplicantes e todas as vagas
    sim_scores = similarity_matrix[applicant_index]
    top_indices = sim_scores.argsort()[::-1][:top_n] 

    # Looping through the top job recommendations based on similarity score to get job details
    # Iterando sobre as recomendações para obter informações das vagas
    for j in top_indices:
        job_id = job_ids[j]
        job = jobs.get(job_id, {})
        job_title = job.get("informacoes_basicas", {}).get("titulo_vaga", "N/A")
        job_area = job.get("perfil_vaga", {}).get("areas_atuacao", "N/A")
        job_skills = job.get("perfil_vaga", {}).get("competencia_tecnicas_e_comportamentais", "N/A")
        job_activities = job.get("perfil_vaga", {}).get("principais_atividades", "N/A")

        # Displaying the job recommendation and details
        # Mostrando os detalhes da recomendação
        print(f"🔹 Job Recommendation: {job_title} (ID: {job_id})")
        print(f"   Similarity Score: {sim_scores[j]:.2f}")  # Display the similarity score for the job
        print(f"   Área: {job_area}")
        print(f"   🔧 Competências: {job_skills[:250]}...")  # Show a snippet of the job's skills required
        print(f"   📋 Atividades: {job_activities[:250]}...\n")  # Show a snippet of the job's activities
        print('--------------------------------------------------------------------')


In [ ]:
show_recommendations_for_applicant(1, top_n=3)

## Machine Learning Step

En:
Now that we have the similarity and the recommendations, we will use Logistic Regression to predict the probability that one applicant will be hired based on the similarity score.

Pt:
Agora que temos a similaridade e as recomendações, usaremos a Regressão Logística para prever a probabilidade de um candidato ser contratado com base na pontuação de similaridade.

In [ ]:
# Creating a dictionary with Job ID, applicat ID and status.
# We are using the file prospects to get the information of which job was filled with which applicant.

# Criando um dicionário com ID da Vaga, ID do Candidato e Status.
# Estamos usando o arquivo de prospects para obter as informações sobre qual vaga foi preenchida com qual candidato.

job_applicant_status = defaultdict(dict)

# job_id → applicant_id → status
for job_id, job_data in prospects.items():
    for prospect in job_data.get("prospects", []):
        applicant_id = prospect["codigo"]
        status = prospect.get("situacao_candidado", "")
        job_applicant_status[job_id][applicant_id] = status

In [ ]:
# Standardizing the status for ML prediction
# Padronizando os status para predição do modelo.

def label_from_status(status):
    status = status.lower()
    if "contratado" in status or "fechado" in status or "encaminhado" in status:
        return 1  # Match
    else:
        return 0  # no Match

In [ ]:
# Creating a merge DF with similarity data and now the status data.
# Crriando um dataframe consolidado com os valores de similaridade e os status obtidos agora.

records = []
TOP_N = 10

for i, applicant_id in enumerate(applicant_ids):
    sim_scores = similarity_matrix[i]
    top_indices = sim_scores.argsort()[::-1][:TOP_N]

    for j in top_indices:
        print(i, len(applicant_ids))
        job_id = job_ids[j]
        sim = sim_scores[j]

        status = job_applicant_status.get(job_id, {}).get(applicant_id, "")
        label = label_from_status(status)

        records.append({
            "applicant_id": applicant_id,
            "job_id": job_id,
            "similarity_score": sim,
            "status": status,
            "label": label
        })

df = pd.DataFrame(records)


In [ ]:
# Counting all the possibility of status before the standadization.
# Contando todas as pssibilidades de status antes da padronização.

df['status'].value_counts()

In [ ]:
# Saving the Df so we save time if running the code again is needed.
# Salvando o dataframe caso precisemos dele novamente.

df.to_pickle('labeled_df')

In [ ]:
# Running the Logistic Regression model
# Rodando o modelo de Regressão Logística

df["binary_label"] = df["label"]

X = df[["similarity_score"]]
y = df["binary_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Running the XGB model
# Rodando o modelo de XGB

from xgboost import XGBClassifier

model = XGBClassifier(
    scale_pos_weight=80000 / 139,  # imbalance ratio
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Because of unbalanced data, we will need to balance our DF with similar proportion of 0 and 1.
# Por conta dos dados desbalanceados, teremos que balancear nosso dataframe como proporção similar de 0 e 1.
from sklearn.utils import resample

df_majority = df[df.label == 0]
df_minority = df[df.label == 1]

df_majority_downsampled = resample(df_majority,
                                   replace=False,
                                   n_samples=len(df_minority) * 3,
                                   random_state=42)

df_balanced = pd.concat([df_majority_downsampled, df_minority])

In [ ]:
# Running once again the Logistic Regression model after the resample
# Rodando o modelo de Regressão logística mais uma vez depois do resample

df_balanced["binary_label"] = df_balanced["label"]

X = df_balanced[["similarity_score"]]
y = df_balanced["binary_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Running once again the XGB model after the resample
# Rodando o modelo de XGB mais uma vez depois do resample

model = XGBClassifier(
    scale_pos_weight=80000 / 139,  # imbalance ratio
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Recreating the job recommendation function but now with the hire probability
# Recriando a função de recomendação de vagas, porém agora com a probilidade de contratação
def show_recommendations_for_applicant(applicant_index, top_n=5, model=None):
    applicant_id = applicant_ids[applicant_index]
    applicant = applicants.get(applicant_id, {})

    # Extract applicant details
    name = applicant.get("infos_basicas", {}).get("nome", "N/A")
    area = applicant.get("informacoes_profissionais", {}).get("area_atuacao", "N/A")
    skills = applicant.get("informacoes_profissionais", {}).get("conhecimentos_tecnicos", "N/A")
    academic = applicant.get("formacao_e_idiomas", {}).get("nivel_academico", "N/A")
    english = applicant.get("formacao_e_idiomas", {}).get("nivel_ingles", "N/A")
    spanish = applicant.get("formacao_e_idiomas", {}).get("nivel_espanhol", "N/A")
    cv_excerpt = applicant.get("cv_pt", "").strip().replace("\n", " ")[:300] + "..."

    # Show applicant info once
    print(f"\n=== 🧑 Applicant: {name} (ID: {applicant_id}) ===")
    print(f"Área de Atuação: {area}")
    print(f"Conhecimentos Técnicos: {skills}")
    print(f"Formação: {academic} | Inglês: {english} | Espanhol: {spanish}")
    print(f"📄 CV (resumo): {cv_excerpt}\n")
    print('\n--------------------------------------------------------------------\n')

    # Job recommendations
    sim_scores = similarity_matrix[applicant_index]
    top_indices = sim_scores.argsort()[::-1][:top_n]

    for j in top_indices:
        job_id = job_ids[j]
        job = jobs.get(job_id, {})
        job_title = job.get("informacoes_basicas", {}).get("titulo_vaga", "N/A")
        job_area = job.get("perfil_vaga", {}).get("areas_atuacao", "N/A")
        job_skills = job.get("perfil_vaga", {}).get("competencia_tecnicas_e_comportamentais", "N/A")
        job_activities = job.get("perfil_vaga", {}).get("principais_atividades", "N/A")

        # Get similarity score
        sim_score = sim_scores[j]

        # Use model to predict hire probability
        hire_prob = None
        if model is not None:
            # Model expects 2D array of features
            hire_prob = model.predict_proba([[sim_score]])[0][1]  # probability of class 1 (hired)

        print(f"🔹 Job Recommendation: {job_title} (ID: {job_id})")
        print(f"   Similarity Score: {sim_score:.2f}")
        if hire_prob is not None:
            print(f"   🤖 Predicted Hire Probability: {hire_prob:.2%}")
        print(f"   Área: {job_area}")
        print(f"   🔧 Competências: {job_skills[:250]}...")
        print(f"   📋 Atividades: {job_activities[:250]}...\n")
        print('--------------------------------------------------------------------')


In [ ]:
show_recommendations_for_applicant(applicant_index=300, top_n=5, model=model)


# Embedding

A biblioteca `sentence-transformers` fornece uma maneira poderosa de transformar texto em **embeddings** (representações vetoriais densas) que capturam o significado semântico do texto. Isso é diferente do método tradicional **TF-IDF**, que se baseia na frequência das palavras e não entende o significado subjacente das palavras e frases.

This code uses the `sentence-transformers` library to convert text into **embeddings** (dense vector representations) that capture the semantic meaning of the text. This is different from the traditional **TF-IDF** method, which relies on word frequencies and does not understand the underlying meaning of words and sentences.

Neste código, estamos utilizando o modelo `paraphrase-multilingual-MiniLM-L12-v2`, que é um modelo **multilíngue** treinado para gerar embeddings de alta qualidade para diferentes idiomas.

In this code, we are using the model `paraphrase-multilingual-MiniLM-L12-v2`, which is a **multilingual** model trained to generate high-quality embeddings for different languages.

O modelo é carregado da seguinte forma:

The model is loaded as follows:

```python
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # good multilingual model
```

Em seguida, usamos o método `encode()` para converter os textos dos **candidatos** e **empregos** em embeddings:

Next, we use the `encode()` method to convert the **applicant** and **job** texts into embeddings:

```python
applicant_embeddings = model.encode(applicant_texts, show_progress_bar=True)
job_embeddings = model.encode(job_texts, show_progress_bar=True)
```

Os embeddings gerados são **vetores numéricos** que representam o significado do texto de uma maneira compacta e semântica. Isso nos permite comparar a semelhança entre os textos de forma eficaz.

The generated embeddings are **numeric vectors** that represent the meaning of the text in a compact and semantic way. This allows us to compare the similarity between texts effectively.

Para calcular a **similaridade** entre os textos dos candidatos e as vagas, usamos o **cosine similarity**:

To calculate the **similarity** between the applicant and job texts, we use **cosine similarity**:

```python
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(applicant_embeddings, job_embeddings)
```

O `cosine similarity` nos dá um valor entre -1 e 1 que representa a **semelhança** entre os vetores, com valores mais próximos de 1 indicando maior semelhança.

Cosine similarity gives us a value between -1 and 1 that represents the **similarity** between the vectors, with values closer to 1 indicating higher similarity.

---

**Comparação entre Embeddings e TF-IDF**

**TF-IDF (Term Frequency-Inverse Document Frequency)** é uma abordagem tradicional para representar texto numericamente. Ele se baseia na **frequência das palavras** em um documento e a importância das palavras em todo o conjunto de dados. O TF-IDF cria **vetores esparsos**, onde cada palavra tem uma posição própria no vetor, e o valor dessa posição é a frequência ponderada da palavra. 

**TF-IDF (Term Frequency-Inverse Document Frequency)** is a traditional approach to numerically represent text. It relies on the **frequency of words** in a document and the importance of words across the dataset. TF-IDF creates **sparse vectors**, where each word has its own position in the vector, and the value of that position is the weighted frequency of the word.

Porém, TF-IDF **não captura o significado semântico** das palavras. Ele trata palavras diferentes como se fossem completamente diferentes, mesmo que tenham significados semelhantes. Por exemplo:

However, TF-IDF **does not capture the semantic meaning** of words. It treats different words as completely different, even if they have similar meanings. For example:

- "Eu amo programar." e "Eu gosto de programar."
- "I love programming." and "I enjoy programming."

No TF-IDF, essas duas frases podem ser representadas de forma bastante diferente, já que "amo" e "gosto" são palavras distintas.

In TF-IDF, these two sentences might be represented quite differently because "love" and "enjoy" are distinct words.

Em contrapartida, os **Embeddings** são vetores densos que representam o **significado** semântico das palavras ou frases. Modelos como o **Word2Vec**, **GloVe** e **Sentence Transformers** convertem palavras ou sentenças em vetores, onde palavras com significados semelhantes são representadas por vetores **próximos** no espaço vetorial. Por exemplo:

On the other hand, **Embeddings** are dense vectors that represent the **semantic meaning** of words or sentences. Models like **Word2Vec**, **GloVe**, and **Sentence Transformers** convert words or sentences into vectors, where words with similar meanings are represented by **close** vectors in the vector space. For example:

- "Amo" e "gosto" teriam vetores próximos, já que ambas as palavras têm significados semelhantes.
- "Love" and "enjoy" would have similar vectors, as both words have similar meanings.

**Vantagens do uso de Embeddings:**

**Advantages of Using Embeddings:**

1. **Entendimento Semântico (Significado Contextual)**: Embeddings conseguem **capturar o significado** das palavras e frases, o que é importante para tarefas como comparação de textos e busca semântica.
   - **Semantic Understanding (Contextual Meaning)**: Embeddings can **capture the meaning** of words and sentences, which is crucial for tasks like text comparison and semantic search.

2. **Sinônimos e Parafraseamento**: Embeddings conseguem entender que palavras com significados semelhantes (sinônimos) ou frases com significados iguais (paráfrases) são semelhantes.
   - **Synonyms and Paraphrasing**: Embeddings can understand that words with similar meanings (synonyms) or sentences with the same meaning (paraphrases) are similar.

3. **Vetores Densos e Compactos**: Diferente do TF-IDF, que cria vetores esparsos (com muitas posições com valor 0), os embeddings criam vetores **densos**, com informações ricas e compactas.
   - **Dense and Compact Vectors**: Unlike TF-IDF, which creates sparse vectors (with many zero-value positions), embeddings create **dense** vectors with rich and compact information.

4. **Melhor Generalização**: Embeddings podem ser **transferidos** entre diferentes conjuntos de dados, aproveitando o treinamento prévio de modelos em grandes corpora de texto, enquanto o TF-IDF é limitado ao conjunto de dados em que é treinado.
   - **Better Generalization**: Embeddings can be **transferred** between different datasets, leveraging the prior training of models on large text corpora, while TF-IDF is limited to the dataset it was trained on.

**Quando Usar TF-IDF**:
- Quando você precisa de uma solução simples e direta e não se importa com o significado semântico do texto.
- Para tarefas como **classificação de documentos** em que a frequência das palavras é mais importante que o significado.
- Quando o conjunto de dados é **menor** e não exige uma compreensão profunda do contexto.

**When to Use TF-IDF**:
- When you need a simple and direct solution and don't care about the semantic meaning of the text.
- For tasks like **document classification** where word frequency is more important than meaning.
- When the dataset is **smaller** and doesn't require a deep understanding of context.

**Quando Usar Embeddings**:
- Quando você precisa capturar o **significado semântico** do texto, como em tarefas de **similaridade de texto**, **respostas a perguntas**, ou **busca semântica**.
- Quando o seu conjunto de dados é **grande** e contém variabilidade linguística (sinônimos, parafraseamento, etc.).
- Quando você precisa lidar com **dados multilíngues** e deseja uma representação consistente do significado em diferentes idiomas.

**When to Use Embeddings**:
- When you need to capture the **semantic meaning** of the text, such as in **text similarity**, **question answering**, or **semantic search** tasks.
- When your dataset is **large** and contains linguistic variability (synonyms, paraphrasing, etc.).
- When you need to handle **multilingual data** and want a consistent representation of meaning across languages.

---

**TF-IDF** é uma boa opção para tarefas simples, enquanto os **Embeddings** são mais poderosos para entender o contexto e a semântica do texto em tarefas mais complexas, como a recomendação de empregos com base no significado de currículos e descrições de vagas.

**TF-IDF** is a good option for simple tasks, while **Embeddings** are more powerful for understanding the context and semantics of text in more complex tasks, such as recommending jobs based on the meaning of resumes and job descriptions.

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # good multilingual model

# Convert preprocessed text
applicant_embeddings = model.encode(applicant_texts, show_progress_bar=True)
job_embeddings = model.encode(job_texts, show_progress_bar=True)

# Similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(applicant_embeddings, job_embeddings)

In [ ]:
# Saving the new similarity matrix with embeddings in case we need it again
# Salvando a nova matriz de similariedade feita com embedding caso precisemos dela novamente.
np.save('similarity_matrix_emb', similarity_matrix)

In [ ]:
show_recommendations_for_applicant(1, top_n=3)

In [ ]:
# Creating once again the combined DF with similarity scores and status from prospects
# Criando novamente o DF com informações de similaride e status do arquivo prospect
records = []
TOP_N = 10

for i, applicant_id in enumerate(applicant_ids):
    sim_scores = similarity_matrix[i]
    top_indices = sim_scores.argsort()[::-1][:TOP_N]

    for j in top_indices:
        print(i, len(applicant_ids))
        job_id = job_ids[j]
        sim = sim_scores[j]

        status = job_applicant_status.get(job_id, {}).get(applicant_id, "")
        label = label_from_status(status)

        records.append({
            "applicant_id": applicant_id,
            "job_id": job_id,
            "similarity_score": sim,
            "status": status,
            "label": label
        })

df = pd.DataFrame(records)

In [ ]:
# Saving the new DF with embeddings in case we need it again
# Salvando o novo dataframe feito com embedding caso precisemos dele novamente.
df.to_pickle('labeled_df_emb')

In [ ]:
# Balancing the new df
# Balanceando o novo dataframe

df_majority = df[df.label == 0]
df_minority = df[df.label == 1]

df_majority_downsampled = resample(df_majority,
                                   replace=False,
                                   n_samples=len(df_minority) * 3,
                                   random_state=42)

df_balanced = pd.concat([df_majority_downsampled, df_minority])

In [ ]:
# Running the Logistic Regression model again with the embedding's data
# Rodando o modelo de Regressão Logística novamente com os novos dados do embedding

df_balanced["binary_label"] = df_balanced["label"]

X = df_balanced[["similarity_score"]]
y = df_balanced["binary_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Running the XGB model again with the embedding's data
# Rodando o modelo de XGB novamente com os novos dados do embedding

model = XGBClassifier(
    scale_pos_weight=80000 / 139,  # imbalance ratio
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
show_recommendations_for_applicant(applicant_index=300, top_n=5, model=model)